# Binary Classification Using the ChestX-ray14 Dataset

## Step 0: Install PyHealth

In [1]:
!rm -rf PyHealth
!git clone https://github.com/EricSchrock/PyHealth.git
%cd PyHealth
!git checkout ChestX-ray14
!pip install -e .

Cloning into 'PyHealth'...
remote: Enumerating objects: 8101, done.
remote: Counting objects: 100% (1761/1761), done.
remote: Compressing objects: 100% (512/512), done.
remote: Total 8101 (delta 1555), reused 1251 (delta 1249), pack-reused 6340 (from 2)
Receiving objects: 100% (8101/8101), 113.88 MiB | 26.69 MiB/s, done.
Resolving deltas: 100% (5242/5242), done.
/content/PyHealth
Branch 'ChestX-ray14' set up to track remote branch 'ChestX-ray14' from 'origin'.
Switched to a new branch 'ChestX-ray14'
Obtaining file:///content/PyHealth
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.0 MB/s eta 0:00:00
     ━━━

## Step 1: Load Dataset

In [1]:
from pyhealth.datasets import ChestXray14Dataset

dataset = ChestXray14Dataset(download=True, partial=True)
dataset.stats()

INFO:pyhealth.datasets.chestxray14:Downloading ./images_01.tar.gz...


Checking MD5 checksum for ./images_01.tar.gz...


INFO:pyhealth.datasets.chestxray14:Checking MD5 checksum for ./images_01.tar.gz...


Extracting ./images_01.tar.gz...


INFO:pyhealth.datasets.chestxray14:Extracting ./images_01.tar.gz...


...

INFO:pyhealth.datasets.base_dataset:Scanning table: chestxray14 from /content/chestxray14-metadata-pyhealth.csv


INFO:pyhealth.datasets.base_dataset:Collecting global event dataframe...


Collected dataframe with shape: (4999, 26)


INFO:pyhealth.datasets.base_dataset:Collected dataframe with shape: (4999, 26)


Dataset: ChestX-ray14
Dev mode: False
Number of patients: 1335
Number of events: 4999


## Step 2: Define Task

In [2]:
from pyhealth.tasks import ChestXray14BinaryClassification

task = ChestXray14BinaryClassification(disease="infiltration")
samples = dataset.set_task(task)

Setting task ChestXray14BinaryClassification for ChestX-ray14 base dataset...


INFO:pyhealth.datasets.base_dataset:Setting task ChestXray14BinaryClassification for ChestX-ray14 base dataset...


Generating samples with 1 worker(s)...


INFO:pyhealth.datasets.base_dataset:Generating samples with 1 worker(s)...
Generating samples for ChestXray14BinaryClassification with 1 worker: 100%|██████████| 1335/1335 [00:00<00:00, 1770.85it/s]

Label label vocab: {0: 0, 1: 1}



INFO:pyhealth.processors.label_processor:Label label vocab: {0: 0, 1: 1}
Processing samples: 100%|██████████| 4999/4999 [01:22<00:00, 60.94it/s]

Generated 4999 samples for task ChestXray14BinaryClassification



INFO:pyhealth.datasets.base_dataset:Generated 4999 samples for task ChestXray14BinaryClassification


In [3]:
from pyhealth.datasets import get_dataloader, split_by_sample

train_dataset, val_dataset, test_dataset = split_by_sample(samples, [0.7, 0.1, 0.2])

train_loader = get_dataloader(train_dataset, batch_size=16, shuffle=True)
val_loader = get_dataloader(val_dataset, batch_size=16, shuffle=False)
test_loader = get_dataloader(test_dataset, batch_size=16, shuffle=False)

## Step 3: Define Model

In [4]:
from pyhealth.models import CNN

model = CNN(dataset=samples)

/content/PyHealth/pyhealth/metrics/calibration.py:122: SyntaxWarning: invalid escape sequence '\c'
  accuracy of 1. Thus, the ECE is :math:`\\frac{1}{3} \cdot 0.49 + \\frac{2}{3}\cdot 0.3=0.3633`.


## Step 4: Train Model

In [5]:
from pyhealth.trainer import Trainer

trainer = Trainer(model=model)
trainer.train(train_dataloader=train_loader, val_dataloader=val_loader, epochs=1)

CNN(
  (embedding_model): EmbeddingModel(embedding_layers=ModuleDict())
  (cnn): ModuleDict(
    (image): CNNLayer(
      (cnn): ModuleList(
        (0): CNNBlock(
          (conv1): Sequential(
            (0): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (conv2): Sequential(
            (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (downsample): Sequential(
            (0): Conv2d(1, 128, kernel_size=(1, 1), stride=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (relu): ReLU()
        )
      )
      (pooling): AdaptiveAvgPool2d(output_size=1)
    )
  )
  (fc): Linear(in_features=128, out_features=1

INFO:pyhealth.trainer:CNN(
  (embedding_model): EmbeddingModel(embedding_layers=ModuleDict())
  (cnn): ModuleDict(
    (image): CNNLayer(
      (cnn): ModuleList(
        (0): CNNBlock(
          (conv1): Sequential(
            (0): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (conv2): Sequential(
            (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (downsample): Sequential(
            (0): Conv2d(1, 128, kernel_size=(1, 1), stride=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (relu): ReLU()
        )
      )
      (pooling): AdaptiveAvgPool2d(output_size=1)
    )
  )
  (fc): Linear(in_featur

Metrics: None


INFO:pyhealth.trainer:Metrics: None


Device: cuda


...

INFO:pyhealth.trainer:roc_auc: 0.6607


f1: 0.0000


INFO:pyhealth.trainer:f1: 0.0000


loss: 0.4239


INFO:pyhealth.trainer:loss: 0.4239


## Step 5: Evaluate Model

In [6]:
trainer.evaluate(test_loader)

Evaluation: 100%|██████████| 63/63 [00:04<00:00, 14.34it/s]


{'pr_auc': 0.2584620467432758,
 'roc_auc': 0.6359064935064935,
 'f1': 0.0,
 'loss': 0.4518731695318979}